In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import StandardScaler

#워닝 메시지
import warnings
warnings.filterwarnings('ignore')


In [3]:
train = pd.read_csv('./Dataset/Undersampling/OSS_0.33_train.csv', encoding='euc-kr')
test = pd.read_csv('./Dataset/Undersampling/OSS_0.33_test.csv', encoding='euc-kr')

In [4]:
X_train_int=train[['총자산대비잉여현금흐름', 'log자산총계', '순운전자본회전률', '총자본증가율', '자기자본구성비율', '총자본회전률',
       '자기자본회전률', '자기자본증가율', '유동자산회전률', '총자산대비현금흐름', '총자본투자효율', '총자본순이익률']]

X_test_int=test[['총자산대비잉여현금흐름', 'log자산총계', '순운전자본회전률', '총자본증가율', '자기자본구성비율', '총자본회전률',
       '자기자본회전률', '자기자본증가율', '유동자산회전률', '총자산대비현금흐름', '총자본투자효율', '총자본순이익률']]


In [5]:
X_train = train.drop('t-1감사의견코드',axis=1)
y_train = train[['t-1감사의견코드']]

X_test = test.drop('t-1감사의견코드',axis=1)
y_test = test[['t-1감사의견코드']]

---

- logistic

In [180]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_logistic_regression(X_train, y_train, X_test, y_test):
    # Stratified 5-fold 교차검증 설정
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

    # 최적 하이퍼파라미터 설정 (여기서는 고정값으로 사용)
    best_params = {'C': 0.07, 'penalty': 'l2'}

    # 각 fold 별 평가 지표를 저장할 리스트 초기화
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []
    confusion_matrix_list = []

    best_f1_score = 0
    best_model = None

    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train), 1):
        X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_test_fold, y_test_fold = X_train.iloc[test_idx], y_train.iloc[test_idx]

        # 로지스틱 회귀 모델 초기화
        model = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'], n_jobs=-1)

        # 모델 학습
        model.fit(X_train_fold, y_train_fold)

        # 테스트 데이터에 대한 예측
        y_pred = model.predict(X_test_fold)

        # 평가 지표 계산
        accuracy = accuracy_score(y_test_fold, y_pred)
        precision = precision_score(y_test_fold, y_pred)
        recall = recall_score(y_test_fold, y_pred)
        f1 = f1_score(y_test_fold, y_pred)
        conf_matrix = confusion_matrix(y_test_fold, y_pred)

        # 각 fold 별 평가 지표를 리스트에 추가
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_score_list.append(f1)
        confusion_matrix_list.append(conf_matrix)

        print(f"Fold {fold_idx}")
        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 score: {f1}")
        print("Confusion Matrix:")
        print(conf_matrix)
        print("------------------------------")

        # 가장 좋은 f1-score 값을 가진 모델을 저장
        if f1 > best_f1_score:
            best_f1_score = f1
            best_model = model

    # 가장 좋은 f1-score 값을 가진 모델로 최종 예측 수행
    y_pred_final = best_model.predict(X_test)

    # 평가 지표 계산
    accuracy_final = accuracy_score(y_test, y_pred_final)
    precision_final = precision_score(y_test, y_pred_final)
    recall_final = recall_score(y_test, y_pred_final)
    f1_final = f1_score(y_test, y_pred_final)
    conf_matrix_final = confusion_matrix(y_test, y_pred_final)

    print("Final Test Results")
    print(f"Accuracy: {accuracy_final}")
    print(f"Precision: {precision_final}")
    print(f"Recall: {recall_final}")
    print(f"F1 score: {f1_final}")
    print("Confusion Matrix:")
    print(conf_matrix_final)

    return accuracy_final, precision_final, recall_final, f1_final


In [181]:
evaluate_logistic_regression(X_train_int, y_train, X_test_int, y_test)

Fold 1
Accuracy: 0.8361757105943153
Precision: 0.7393538913362702
Recall: 0.5244791666666667
F1 score: 0.6136502132845826
Confusion Matrix:
[[5465  355]
 [ 913 1007]]
------------------------------
Fold 2
Accuracy: 0.8301033591731266
Precision: 0.7214912280701754
Recall: 0.5137948984903696
F1 score: 0.6001824262693828
Confusion Matrix:
[[5438  381]
 [ 934  987]]
------------------------------
Fold 3
Accuracy: 0.8368217054263566
Precision: 0.746996996996997
Recall: 0.5179593961478397
F1 score: 0.6117430064555794
Confusion Matrix:
[[5482  337]
 [ 926  995]]
------------------------------
Fold 4
Accuracy: 0.839901796097687
Precision: 0.7409766454352441
Recall: 0.5453125
F1 score: 0.6282628262826282
Confusion Matrix:
[[5453  366]
 [ 873 1047]]
------------------------------
Fold 5
Accuracy: 0.832536503424215
Precision: 0.7342342342342343
Recall: 0.509375
F1 score: 0.6014760147601476
Confusion Matrix:
[[5465  354]
 [ 942  978]]
------------------------------
Final Test Results
Accuracy: 0.8

(0.837082540231874, 0.7481108312342569, 0.5177824267782427, 0.6119925819080982)

---

- Randomforest

In [31]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_random_forest(X_train, y_train, X_test, y_test):
    # Stratified 5-fold 교차검증 설정
    cv = StratifiedKFold(n_splits=7, shuffle=True, random_state=0)

    # 최적 하이퍼파라미터 설정 (여기서는 고정값으로 사용)
    best_params = {'max_depth': 9, 'min_samples_leaf': 20, 'max_features':'auto', 'n_estimators':200}

    # 각 fold 별 평가 지표를 저장할 리스트 초기화
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []
    confusion_matrix_list = []

    best_f1_score = 0
    best_model = None

    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train), 1):
        X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_test_fold, y_test_fold = X_train.iloc[test_idx], y_train.iloc[test_idx]

        # 랜덤 포레스트 모델 초기화
        model = RandomForestClassifier(max_depth=best_params['max_depth'],
                                       min_samples_leaf=best_params['min_samples_leaf'],
                                       n_estimators=best_params['n_estimators'],
                                       max_features= best_params['max_features'],
                                       random_state=0)

        # 모델 학습
        model.fit(X_train_fold, y_train_fold)

        # 테스트 데이터에 대한 예측 확률 얻기
        probabilities = model.predict_proba(X_test_fold)

        # threshold를 0.4로 설정하여 예측 클래스를 조정
        threshold = 0.43
        predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)

        # 평가 지표 계산
        accuracy = accuracy_score(y_test_fold, predicted_classes)
        precision = precision_score(y_test_fold, predicted_classes)
        recall = recall_score(y_test_fold, predicted_classes)
        f1 = f1_score(y_test_fold, predicted_classes)
        conf_matrix = confusion_matrix(y_test_fold, predicted_classes)

        # 각 fold 별 평가 지표를 리스트에 추가
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_score_list.append(f1)
        confusion_matrix_list.append(conf_matrix)

        print(f"Fold {fold_idx}")
        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 score: {f1}")
        print("Confusion Matrix:")
        print(conf_matrix)
        print("------------------------------")

        # 가장 좋은 f1-score 값을 가진 모델을 저장
        if f1 > best_f1_score:
            best_f1_score = f1
            best_model = model

    # 가장 좋은 f1-score 값을 가진 모델로 최종 예측 수행
    probabilities_final = best_model.predict_proba(X_test)
    y_pred_final = (probabilities_final[:, 1] > threshold).astype(np.int)

    # 평가 지표 계산
    accuracy_final = accuracy_score(y_test, y_pred_final)
    precision_final = precision_score(y_test, y_pred_final)
    recall_final = recall_score(y_test, y_pred_final)
    f1_final = f1_score(y_test, y_pred_final)
    conf_matrix_final = confusion_matrix(y_test, y_pred_final)

    print("Final Test Results")
    print(f"Accuracy: {accuracy_final}")
    print(f"Precision: {precision_final}")
    print(f"Recall: {recall_final}")
    print(f"F1 score: {f1_final}")
    print("Confusion Matrix:")
    print(conf_matrix_final)

    return accuracy_list, precision_list, recall_list, f1_score_list


In [32]:
evaluate_random_forest(X_train_int, y_train, X_test_int, y_test)

Fold 1
Accuracy: 0.8934707903780069
Precision: 0.7758985200845666
Recall: 0.8024781341107872
F1 score: 0.7889645288427086
Confusion Matrix:
[[3839  318]
 [ 271 1101]]
------------------------------
Fold 2
Accuracy: 0.8913004159884247
Precision: 0.7675225537820958
Recall: 0.8061224489795918
F1 score: 0.7863490934944898
Confusion Matrix:
[[3822  335]
 [ 266 1106]]
------------------------------
Fold 3
Accuracy: 0.8901953690303908
Precision: 0.7776162790697675
Recall: 0.7804522246535376
F1 score: 0.779031670913724
Confusion Matrix:
[[3851  306]
 [ 301 1070]]
------------------------------
Fold 4
Accuracy: 0.8925470332850941
Precision: 0.7809110629067245
Recall: 0.787746170678337
F1 score: 0.7843137254901961
Confusion Matrix:
[[3854  303]
 [ 291 1080]]
------------------------------
Fold 5
Accuracy: 0.8939942112879884
Precision: 0.7688098495212038
Recall: 0.8192419825072886
F1 score: 0.7932251235003529
Confusion Matrix:
[[3818  338]
 [ 248 1124]]
------------------------------
Fold 6
Accur

([0.8934707903780069,
  0.8913004159884247,
  0.8901953690303908,
  0.8925470332850941,
  0.8939942112879884,
  0.8869392185238785,
  0.8880246020260492],
 [0.7758985200845666,
  0.7675225537820958,
  0.7776162790697675,
  0.7809110629067245,
  0.7688098495212038,
  0.7588357588357588,
  0.7638402242466713],
 [0.8024781341107872,
  0.8061224489795918,
  0.7804522246535376,
  0.787746170678337,
  0.8192419825072886,
  0.7981049562682215,
  0.7944606413994169],
 [0.7889645288427086,
  0.7863490934944898,
  0.779031670913724,
  0.7843137254901961,
  0.7932251235003529,
  0.77797513321492,
  0.7788495891389782])

---
# LGBM

In [67]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_lgbm(X_train, y_train, X_test, y_test, k_fold=5):
    # Stratified k-fold 교차검증 설정
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    # LGBM 하이퍼파라미터 설정 (여기서는 고정값으로 사용)
    params = {
        'max_depth': 20,
        'min_child_samples': 10,
        'n_estimators':180,
        'learning_rate': 0.1,
        'objective': 'binary',
        'random_state': 0
    }

    # 각 fold 별 평가 지표를 저장할 리스트 초기화
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []
    confusion_matrix_list = []

    best_f1_score = 0
    best_model = None

    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train), 1):
        X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_test_fold, y_test_fold = X_train.iloc[test_idx], y_train.iloc[test_idx]

        # LGBM 모델 초기화
        model = lgb.LGBMClassifier(**params)

        # 모델 학습
        model.fit(X_train_fold, y_train_fold)

        # 테스트 데이터에 대한 예측 확률 얻기
        probabilities = model.predict_proba(X_test_fold)

        # threshold를 0.4로 설정하여 예측 클래스를 조정
        threshold = 0.5
        predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)

        # 평가 지표 계산
        accuracy = accuracy_score(y_test_fold, predicted_classes)
        precision = precision_score(y_test_fold, predicted_classes)
        recall = recall_score(y_test_fold, predicted_classes)
        f1 = f1_score(y_test_fold, predicted_classes)
        conf_matrix = confusion_matrix(y_test_fold, predicted_classes)

        # 각 fold 별 평가 지표를 리스트에 추가
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_score_list.append(f1)
        confusion_matrix_list.append(conf_matrix)

        print(f"Fold {fold_idx}")
        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 score: {f1}")
        print("Confusion Matrix:")
        print(conf_matrix)
        print("------------------------------")

        # 가장 좋은 f1-score 값을 가진 모델을 저장
        if f1 > best_f1_score:
            best_f1_score = f1
            best_model = model

    # 가장 좋은 f1-score 값을 가진 모델로 최종 예측 수행
    probabilities_final = best_model.predict_proba(X_test)
    y_pred_final = (probabilities_final[:, 1] > threshold).astype(np.int)

    # 평가 지표 계산
    accuracy_final = accuracy_score(y_test, y_pred_final)
    precision_final = precision_score(y_test, y_pred_final)
    recall_final = recall_score(y_test, y_pred_final)
    f1_final = f1_score(y_test, y_pred_final)
    conf_matrix_final = confusion_matrix(y_test, y_pred_final)

    print("Final Test Results")
    print(f"Accuracy: {accuracy_final}")
    print(f"Precision: {precision_final}")
    print(f"Recall: {recall_final}")
    print(f"F1 score: {f1_final}")
    print("Confusion Matrix:")
    print(conf_matrix_final)

    return accuracy_list, precision_list, recall_list, f1_score_list


In [68]:
evaluate_lgbm(X_train_int, y_train, X_test_int, y_test)

Fold 1
Accuracy: 0.891343669250646
Precision: 0.7972451790633609
Recall: 0.7536458333333333
F1 score: 0.7748326639892905
Confusion Matrix:
[[5452  368]
 [ 473 1447]]
------------------------------
Fold 2
Accuracy: 0.8988372093023256
Precision: 0.8105895196506551
Recall: 0.7730348776678813
F1 score: 0.7913669064748202
Confusion Matrix:
[[5472  347]
 [ 436 1485]]
------------------------------
Fold 3
Accuracy: 0.8939276485788114
Precision: 0.8042035398230089
Recall: 0.7568974492451848
F1 score: 0.7798337355859479
Confusion Matrix:
[[5465  354]
 [ 467 1454]]
------------------------------
Fold 4
Accuracy: 0.8953353146401344
Precision: 0.7942735949098622
Recall: 0.7802083333333333
F1 score: 0.7871781397792957
Confusion Matrix:
[[5431  388]
 [ 422 1498]]
------------------------------
Fold 5
Accuracy: 0.8944308050135676
Precision: 0.7992403689636463
Recall: 0.7671875
F1 score: 0.7828859952165825
Confusion Matrix:
[[5449  370]
 [ 447 1473]]
------------------------------
Final Test Results
A

([0.891343669250646,
  0.8988372093023256,
  0.8939276485788114,
  0.8953353146401344,
  0.8944308050135676],
 [0.7972451790633609,
  0.8105895196506551,
  0.8042035398230089,
  0.7942735949098622,
  0.7992403689636463],
 [0.7536458333333333,
  0.7730348776678813,
  0.7568974492451848,
  0.7802083333333333,
  0.7671875],
 [0.7748326639892905,
  0.7913669064748202,
  0.7798337355859479,
  0.7871781397792957,
  0.7828859952165825])

---

In [211]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import numpy as np

def perform_lgbm_grid_search(X_train, y_train, k_fold=5):
    # Stratified k-fold 교차검증 설정
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    # LGBM 하이퍼파라미터 후보 리스트 설정
    param_grid = {
        'max_depth': [16],
        'n_estimators': [220],
        'learning_rate': [0.08],
        'objective': ['binary'],
        'random_state': [0],
        'min_child_samples' :[20]
    }

    # LGBM 모델 초기화
    model = lgb.LGBMClassifier()

    # 그리드 서치 설정
    grid_search = GridSearchCV(model, param_grid, scoring='recall', cv=cv, verbose=1, n_jobs=-1)

    # 모델 학습 및 튜닝
    grid_search.fit(X_train, y_train)

    # 최적 하이퍼파라미터 출력
    print("Best Hyperparameters:", grid_search.best_params_)

    # 평균 평가 지표 계산
    mean_f1_score = np.mean(grid_search.cv_results_['mean_test_score'])
    print("Mean F1 Score:", mean_f1_score)

    return grid_search.best_params_, mean_f1_score


In [212]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_lgbm_with_best_params(X_train, y_train, X_test, y_test, best_params, k_fold=5):
    # Stratified k-fold 교차검증 설정
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    # LGBM 모델 초기화
    model = lgb.LGBMClassifier(**best_params)

    # 각 fold 별 평가 지표를 저장할 리스트 초기화
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []
    confusion_matrix_list = []

    best_f1_score = 0
    best_model = None

    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train), 1):
        X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_test_fold, y_test_fold = X_train.iloc[test_idx], y_train.iloc[test_idx]

        # 모델 학습
        model.fit(X_train_fold, y_train_fold)

        # 테스트 데이터에 대한 예측 확률 얻기
        probabilities = model.predict_proba(X_test_fold)

        # threshold를 0.4로 설정하여 예측 클래스를 조정
        threshold = 0.4
        predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)

        # 평가 지표 계산
        accuracy = accuracy_score(y_test_fold, predicted_classes)
        precision = precision_score(y_test_fold, predicted_classes)
        recall = recall_score(y_test_fold, predicted_classes)
        f1 = f1_score(y_test_fold, predicted_classes)
        conf_matrix = confusion_matrix(y_test_fold, predicted_classes)

        # 각 fold 별 평가 지표를 리스트에 추가
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_score_list.append(f1)
        confusion_matrix_list.append(conf_matrix)

        print(f"Fold {fold_idx}")
        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 score: {f1}")
        print("Confusion Matrix:")
        print(conf_matrix)
        print("------------------------------")

        # 가장 좋은 f1-score 값을 가진 모델을 저장
        if f1 > best_f1_score:
            best_f1_score = f1
            best_model = model

    # 가장 좋은 f1-score 값을 가진 모델로 최종 예측 수행
    probabilities_final = best_model.predict_proba(X_test)
    y_pred_final = (probabilities_final[:, 1] > threshold).astype(np.int)

    # 평가 지표 계산
    accuracy_final = accuracy_score(y_test, y_pred_final)
    precision_final = precision_score(y_test, y_pred_final)
    recall_final = recall_score(y_test, y_pred_final)
    f1_final = f1_score(y_test, y_pred_final)
    conf_matrix_final = confusion_matrix(y_test, y_pred_final)

    print("Final Test Results")
    print(f"Accuracy: {accuracy_final}")
    print(f"Precision: {precision_final}")
    print(f"Recall: {recall_final}")
    print(f"F1 score: {f1_final}")
    print("Confusion Matrix:")
    print(conf_matrix_final)

    return accuracy_list, precision_list, recall_list, f1_score_list


In [213]:
best_params, mean_f1_score = perform_lgbm_grid_search(X_train, y_train, k_fold=7)
evaluate_lgbm_with_best_params(X_train_int, y_train, X_test_int, y_test, best_params, k_fold=7)

Fitting 7 folds for each of 1 candidates, totalling 7 fits
Best Hyperparameters: {'learning_rate': 0.08, 'max_depth': 16, 'min_child_samples': 20, 'n_estimators': 220, 'objective': 'binary', 'random_state': 0}
Mean F1 Score: 0.7611923794212903
Fold 1
Accuracy: 0.8947368421052632
Precision: 0.7777777777777778
Recall: 0.8061224489795918
F1 score: 0.7916964924838941
Confusion Matrix:
[[3841  316]
 [ 266 1106]]
------------------------------
Fold 2
Accuracy: 0.8903960933260987
Precision: 0.7656033287101248
Recall: 0.8046647230320699
F1 score: 0.7846481876332622
Confusion Matrix:
[[3819  338]
 [ 268 1104]]
------------------------------
Fold 3
Accuracy: 0.8914616497829233
Precision: 0.7767408470926059
Recall: 0.7892049598832969
F1 score: 0.7829232995658467
Confusion Matrix:
[[3846  311]
 [ 289 1082]]
------------------------------
Fold 4
Accuracy: 0.8974312590448625
Precision: 0.7896253602305475
Recall: 0.799416484318016
F1 score: 0.794490757520841
Confusion Matrix:
[[3865  292]
 [ 275 1096

([0.8947368421052632,
  0.8903960933260987,
  0.8914616497829233,
  0.8974312590448625,
  0.8959840810419681,
  0.8873010130246021,
  0.8873010130246021],
 [0.7777777777777778,
  0.7656033287101248,
  0.7767408470926059,
  0.7896253602305475,
  0.7712729748127978,
  0.7559808612440191,
  0.7617051013277428],
 [0.8061224489795918,
  0.8046647230320699,
  0.7892049598832969,
  0.799416484318016,
  0.825801749271137,
  0.8061224489795918,
  0.7944606413994169],
 [0.7916964924838941,
  0.7846481876332622,
  0.7829232995658467,
  0.794490757520841,
  0.7976064765927491,
  0.7802469135802469,
  0.7777381377095968])

---

# XGBM

In [178]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import numpy as np

def perform_xgb_grid_search(X_train, y_train, k_fold=5):
    # Stratified k-fold 교차검증 설정
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    # XGBoost 하이퍼파라미터 후보 리스트 설정
    param_grid = {
        'max_depth': [6, 8, 10],
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.3],
        'objective': ['binary:logistic'],
        'random_state': [0],
    }

    # XGBoost 모델 초기화
    model = xgb.XGBClassifier()

    # 그리드 서치 설정
    grid_search = GridSearchCV(model, param_grid, scoring='f1', cv=cv, verbose=1, n_jobs=-1)

    # 모델 학습 및 튜닝
    grid_search.fit(X_train, y_train)

    # 최적 하이퍼파라미터 출력
    print("Best Hyperparameters:", grid_search.best_params_)

    # 평균 평가 지표 계산
    mean_f1_score = np.mean(grid_search.cv_results_['mean_test_score'])
    print("Mean F1 Score:", mean_f1_score)

    return grid_search.best_params_, mean_f1_score


import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_xgb_with_best_params(X_train, y_train, X_test, y_test, best_params, k_fold=5):
    # Stratified k-fold 교차검증 설정
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    # XGBoost 모델 초기화
    model = xgb.XGBClassifier(**best_params)

    # 각 fold 별 평가 지표를 저장할 리스트 초기화
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []
    confusion_matrix_list = []

    best_f1_score = 0
    best_model = None

    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train), 1):
        X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_test_fold, y_test_fold = X_train.iloc[test_idx], y_train.iloc[test_idx]

        # 모델 학습
        model.fit(X_train_fold, y_train_fold)

        # 테스트 데이터에 대한 예측 확률 얻기
        probabilities = model.predict_proba(X_test_fold)

        # threshold를 0.4로 설정하여 예측 클래스를 조정
        threshold = 0.4
        predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)

        # 평가 지표 계산
        accuracy = accuracy_score(y_test_fold, predicted_classes)
        precision = precision_score(y_test_fold, predicted_classes)
        recall = recall_score(y_test_fold, predicted_classes)
        f1 = f1_score(y_test_fold, predicted_classes)
        conf_matrix = confusion_matrix(y_test_fold, predicted_classes)

        # 각 fold 별 평가 지표를 리스트에 추가
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_score_list.append(f1)
        confusion_matrix_list.append(conf_matrix)

        print(f"Fold {fold_idx}")
        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 score: {f1}")
        print("Confusion Matrix:")
        print(conf_matrix)
        print("------------------------------")

        # 가장 좋은 f1-score 값을 가진 모델을 저장
        if f1 > best_f1_score:
            best_f1_score = f1
            best_model = model

    # 가장 좋은 f1-score 값을 가진 모델로 최종 예측 수행
    probabilities_final = best_model.predict_proba(X_test)
    y_pred_final = (probabilities_final[:, 1] > threshold).astype(np.int)

    # 평가 지표 계산
    accuracy_final = accuracy_score(y_test, y_pred_final)
    precision_final = precision_score(y_test, y_pred_final)
    recall_final = recall_score(y_test, y_pred_final)
    f1_final = f1_score(y_test, y_pred_final)
    conf_matrix_final = confusion_matrix(y_test, y_pred_final)

    print("Final Test Results")
    print(f"Accuracy: {accuracy_final}")
    print(f"Precision: {precision_final}")
    print(f"Recall: {recall_final}")
    print(f"F1 score: {f1_final}")
    print("Confusion Matrix:")
    print(conf_matrix_final)

    return accuracy_list, precision_list, recall_list, f1_score_list





In [179]:
best_params, mean_f1_score = perform_xgb_grid_search(X_train, y_train, k_fold=5)
evaluate_xgb_with_best_params(X_train_int, y_train, X_test_int, y_test, best_params, k_fold=7)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


KeyboardInterrupt: 